Обработка аугментаций из RoboFlow и формирование аугментаций с антиклассами

In [33]:
import pandas as pd
from pathlib import Path
import os
import cv2
import numpy as np
import albumentations as A
from PIL import Image, ImageFile
from tqdm import tqdm

In [13]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Обработка аугментаций с новыми терминалами

Замена id 0 на 2

In [2]:
path_yolo_annot = Path(r'..\data\terminal_object_detection_data\yolo_annot\new_terminals_augm')

In [5]:
def replace_ids_with_2(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        data = line.split()
        if len(data) > 0:
            # Replace the id (class label) with "2"
            data[0] = "2"
            new_line = ' '.join(data)
            new_lines.append(new_line)

    with open(file_path, 'w') as file:
        file.writelines(new_lines)

In [6]:
for filename in os.listdir(path_yolo_annot):
    labelname = path_yolo_annot / filename
    replace_ids_with_2(labelname)

# Формирование аугментаций с антиклассами

In [8]:
path_photos = Path(r'..\data\terminal_object_detection_data\photos')

In [9]:
os.listdir(path_photos)

['new_terminals',
 'new_terminals_augm',
 'other',
 'pc_screens',
 'rack_new_terminal',
 'receipts',
 'stickers',
 'terminals',
 'terminals_augm',
 'terminals_receipt',
 'terminals_side_view',
 'wrong_photos_of_terminals']

In [10]:
anticlass_folders = [
    'other',
    'pc_screens',
    'rack_new_terminal',
    'receipts',
    'stickers',
    'terminals_side_view'
]

In [35]:
trans_contrast = A.RandomBrightnessContrast(
        always_apply=True,
        contrast_limit=(-.6, -.6),
        p=1
    )
trans_spat = A.Spatter(
        always_apply=True, mean=-0.79,
        std=13.16, p=1
    )

trans_clache = A.CLAHE(
        always_apply=True,
        p=1
    )

trans_emboss = A.Emboss(
    always_apply=True,
    p=1
)

trans_sharp = A.Sharpen(
    always_apply=True,
    p=1
)

trans_gauss_noise = A.GaussNoise(
    always_apply=True,
    p=1
)

trans_hue_sat = A.HueSaturationValue(
    always_apply=True,
    hue_shift_limit=(-20, 20),
    sat_shift_limit=(-30, 30),
    val_shift_limit=(-20, 20),
    p=1
)

trans_color_jitter = A.ColorJitter(
    always_apply=True,
    brightness=0.2,
    contrast=0.2,
    saturation=0.2,
    hue=0.2,
    p=1
)

trans_random_crop = A.RandomCrop(
    always_apply=True,
    height=640,
    width=640,
    p=1
)

In [41]:
for anticlass_folder in anticlass_folders:
    augm_folder = anticlass_folder + '_augm'
    image_folder_augm = path_photos /  augm_folder
    label_folder_augm = Path(image_folder_augm.__str__().replace('photos', 'yolo_annot'))
    if not os.path.exists(image_folder_augm):
        os.makedirs(image_folder_augm)
    if not os.path.exists(label_folder_augm):
        os.makedirs(label_folder_augm)
    for image_filename in tqdm(os.listdir(path_photos / anticlass_folder), desc=" file"):
        full_path_image =  path_photos / anticlass_folder / image_filename
        short_filename = full_path_image.__str__().split('\\')[-1].split('.')[0]
        
        image = np.array(Image.open(full_path_image))

        image_contr = trans_contrast(image=image)['image']
        # image_contr = trans_random_crop(image=image_contr)['image']
        im = Image.fromarray(image_contr)
        im.save(image_folder_augm / (short_filename + '_contr.jpeg'))
        open(label_folder_augm / (short_filename + '_contr.txt'), 'w').close()

        image_spat = trans_spat(image=image)['image']
        # image_spat = trans_random_crop(image=image_spat)['image']
        im = Image.fromarray(image_spat)
        im.save(image_folder_augm / (short_filename + '_spat.jpeg'))
        open(label_folder_augm / (short_filename + '_spat.txt'), 'w').close()

        image_clache = trans_clache(image=image)['image']
        # image_clache = trans_random_crop(image=image_clache)['image']
        im = Image.fromarray(image_clache)
        im.save(image_folder_augm / (short_filename + '_clache.jpeg'))
        open(label_folder_augm / (short_filename + '_clache.txt'), 'w').close()        

        image_emboss = trans_emboss(image=image)['image']
        # image_emboss = trans_random_crop(image=image_emboss)['image']
        im = Image.fromarray(image_emboss)
        im.save(image_folder_augm / (short_filename + '_emboss.jpeg'))
        open(label_folder_augm / (short_filename + '_emboss.txt'), 'w').close() 

        image_sharp = trans_sharp(image=image)['image']
        # image_sharp = trans_random_crop(image=image_sharp)['image']
        im = Image.fromarray(image_sharp)
        im.save(image_folder_augm / (short_filename + '_sharp.jpeg'))
        open(label_folder_augm / (short_filename + '_sharp.txt'), 'w').close() 

        image_gauss_noise = trans_gauss_noise(image=image)['image']
        # image_gauss_noise = trans_random_crop(image=image_gauss_noise)['image']
        im = Image.fromarray(image_gauss_noise)
        im.save(image_folder_augm / (short_filename + '_gauss_noise.jpeg'))
        open(label_folder_augm / (short_filename + '_gauss_noise.txt'), 'w').close() 

        image_hue_sat = trans_hue_sat(image=image)['image']
        # image_hue_sat = trans_random_crop(image=image_hue_sat)['image']
        im = Image.fromarray(image_hue_sat)
        im.save(image_folder_augm / (short_filename + '_hue_sat.jpeg'))
        open(label_folder_augm / (short_filename + '_hue_sat.txt'), 'w').close()

        image_color_jitter = trans_color_jitter(image=image)['image']
        # image_color_jitter = trans_random_crop(image=image_color_jitter)['image']
        im = Image.fromarray(image_color_jitter)
        im.save(image_folder_augm / (short_filename + '_color_jitter.jpeg'))
        open(label_folder_augm / (short_filename + '_color_jitter.txt'), 'w').close()
        

 file: 100%|██████████| 428/428 [04:00<00:00,  1.78it/s]
